In [1]:
from model_definitions import *

In [2]:
SEA_Net?

In [74]:
import os
import pandas as pd
import pmlb
from pmlb import fetch_data

# Path to the TSV file
metadata_path = os.path.join(os.path.dirname(pmlb.__file__), 'all_summary_stats.tsv')

# Read the TSV into a DataFrame
metadata = pd.read_csv(metadata_path, sep='\t')

# Filter datasets based on number of records (rows) and other criteria
filtered_datasets = metadata[
    (metadata['n_features'] < 11) &
    (metadata['n_binary_features'] == 0) &
    (metadata['n_categorical_features'] == 0) &
    (metadata['n_continuous_features'] == metadata['n_features'] ) &
    #(metadata['n_instances'] >= 100) &  # Assuming 'n_instances' is the column for the number of records
    (metadata['n_instances'] <= 10000) &
    (metadata['endpoint_type'] == 'continuous') &
    (metadata['task'] == 'regression')
]

#print(filtered_datasets)
filtered_datasets['dataset']


0                    1027_ESL
1                    1028_SWD
2                    1029_LEV
3                    1030_ERA
5        1096_FacultySalaries
                ...          
116    690_visualizing_galaxy
118       706_sleuth_case1202
119       712_chscase_geyser1
155                    banana
270                   titanic
Name: dataset, Length: 64, dtype: object

In [75]:
filtered_datasets

,dataset,n_instances,n_features,n_binary_features,n_categorical_features,n_continuous_features,endpoint_type,n_classes,imbalance,task
0,1027_ESL,488,4,0,0,4,continuous,9.0,0.099363,regression
1,1028_SWD,1000,10,0,0,10,continuous,4.0,0.108291,regression
2,1029_LEV,1000,4,0,0,4,continuous,5.0,0.111245,regression
3,1030_ERA,1000,4,0,0,4,continuous,9.0,0.031251,regression
5,1096_FacultySalaries,50,4,0,0,4,continuous,39.0,0.004063,regression
...,...,...,...,...,...,...,...,...,...,...
116,690_visualizing_galaxy,323,4,0,0,4,continuous,208.0,0.001246,regression
118,706_sleuth_case1202,93,6,0,0,6,continuous,79.0,0.001583,regression
119,712_chscase_geyser1,222,2,0,0,2,continuous,50.0,0.012844,regression
155,banana,5300,2,0,0,2,continuous,2.0,0.010691,regression


In [52]:
import os
import pandas as pd
import pmlb
import requests
import yaml
from pmlb import fetch_data

# Path to the TSV file
metadata_path = os.path.join(os.path.dirname(pmlb.__file__), 'all_summary_stats.tsv')

# Read the TSV into a DataFrame
metadata = pd.read_csv(metadata_path, sep='\t')

# Filter datasets based on initial criteria
initial_filtered_datasets = metadata[
    (metadata['n_features'] < 11) &
    (metadata['n_binary_features'] == 0) &
    (metadata['n_categorical_features'] == 0)
]

# Check each dataset in the initial_filtered_datasets for its description
def has_valid_description(dataset_name):
    url = f"https://raw.githubusercontent.com/EpistasisLab/pmlb/master/datasets/{dataset_name}/metadata.yaml"
    response = requests.get(url)
    if response.status_code != 200:
        return False

    yaml_data = yaml.safe_load(response.text)
    description = yaml_data.get('description', '').strip()
    return description and description.lower() != "none yet. see our contributing guide to help us add one."

# Use ThreadPoolExecutor to speed up the process
with ThreadPoolExecutor(max_workers=10) as executor:
    valid_descriptions = list(executor.map(has_valid_description, initial_filtered_datasets['dataset']))

filtered_datasets = initial_filtered_datasets[valid_descriptions]

#print(filtered_datasets)
len(filtered_datasets)


9

In [53]:
filtered_datasets['dataset']

13            192_vineyard
32              519_vinnie
36              529_pollen
155                 banana
162                   bupa
217                  lupus
219                  magic
223    mfeat_morphological
270                titanic
Name: dataset, dtype: object

In [72]:
import os
import pandas as pd
import pmlb
import requests
import yaml
from pmlb import fetch_data
from concurrent.futures import ThreadPoolExecutor

# Path to the TSV file
metadata_path = os.path.join(os.path.dirname(pmlb.__file__), 'all_summary_stats.tsv')

# Read the TSV into a DataFrame
metadata = pd.read_csv(metadata_path, sep='\t')

# Filter datasets based on number of records (rows) and other criteria
filtered_datasets = metadata[
    (metadata['n_features'] < 20) &
    (metadata['n_binary_features'] == 0) &
    (metadata['n_categorical_features'] == 0) &
    (metadata['n_continuous_features'] == metadata['n_features'] ) &
    (metadata['n_instances'] >= 10) &  
    (metadata['n_instances'] <= 1000000) &
    (metadata['endpoint_type'] == 'cat') &
    (metadata['task'] == 'regression')
]


# Check each dataset in the initial_filtered_datasets for its description
def get_description(dataset_name):
    url = f"https://raw.githubusercontent.com/EpistasisLab/pmlb/master/datasets/{dataset_name}/metadata.yaml"
    response = requests.get(url)
    if response.status_code != 200:
        return None

    yaml_data = yaml.safe_load(response.text)
    description = yaml_data.get('description', '').strip()
    if description and description.lower() != "none yet. see our contributing guide to help us add one.":
        return description
    return None

# Use ThreadPoolExecutor to speed up the process
with ThreadPoolExecutor(max_workers=10) as executor:
    descriptions = list(executor.map(get_description, initial_filtered_datasets['dataset']))

# Filter out None values and datasets without valid descriptions
valid_indices = [index for index, desc in enumerate(descriptions) if desc]
filtered_datasets = initial_filtered_datasets.iloc[valid_indices]

# Create a DataFrame containing only valid descriptions
description_df = pd.DataFrame({
    'dataset': filtered_datasets['dataset'].values,
    'description': [descriptions[i] for i in valid_indices]
})

print(description_df)
(filtered_datasets)


               dataset                                        description
0         192_vineyard  Data from a vineyard on a small island in Lake...
1           519_vinnie  Data on the shooting of Vinnie Johnson of the ...
2           529_pollen  Synthetic pollen dataset generated by David Co...
3               banana  This is an artificial dataset where instances ...
4                 bupa  BUPA medical research on liver disease that mi...
5                lupus  A small dataset meant to compare the time from...
6                magic  particle shower measurements used to classify ...
7  mfeat_morphological  6 morphological features of images of characte...
8              titanic  On the 10th of April 1912, the largest passeng...


,dataset,n_instances,n_features,n_binary_features,n_categorical_features,n_continuous_features,endpoint_type,n_classes,imbalance,task
13,192_vineyard,52,2,0,0,2,continuous,19.0,0.040475,regression
32,519_vinnie,380,2,0,0,2,continuous,16.0,0.030146,regression
36,529_pollen,3848,4,0,0,4,continuous,3784.0,0.000004,regression
155,banana,5300,2,0,0,2,continuous,2.0,0.010691,regression
162,bupa,345,5,0,0,5,categorical,2.0,0.000412,classification
217,lupus,87,3,0,0,3,categorical,2.0,0.038182,classification
219,magic,19020,10,0,0,10,categorical,2.0,0.088055,classification
223,mfeat_morphological,2000,6,0,0,6,categorical,10.0,0.000000,classification
270,titanic,2201,3,0,0,3,continuous,2.0,0.125266,regression


In [59]:
description_df['description'][6]

'particle shower measurements used to classify which type of radiation caused the particle shower.'

In [66]:
filtered_datasets

,dataset,n_instances,n_features,n_binary_features,n_categorical_features,n_continuous_features,endpoint_type,n_classes,imbalance,task
13,192_vineyard,52,2,0,0,2,continuous,19.0,0.040475,regression
32,519_vinnie,380,2,0,0,2,continuous,16.0,0.030146,regression
36,529_pollen,3848,4,0,0,4,continuous,3784.0,0.000004,regression
155,banana,5300,2,0,0,2,continuous,2.0,0.010691,regression
162,bupa,345,5,0,0,5,categorical,2.0,0.000412,classification
217,lupus,87,3,0,0,3,categorical,2.0,0.038182,classification
219,magic,19020,10,0,0,10,categorical,2.0,0.088055,classification
223,mfeat_morphological,2000,6,0,0,6,categorical,10.0,0.000000,classification
270,titanic,2201,3,0,0,3,continuous,2.0,0.125266,regression


In [17]:
# Print all column names
print(metadata.columns)


Index(['dataset', 'n_instances', 'n_features', 'n_binary_features',
       'n_categorical_features', 'n_continuous_features', 'endpoint_type',
       'n_classes', 'imbalance', 'task'],
      dtype='object')


In [20]:
# Function to fetch a dataset by name
def fetch_dataset(row):
    dataset_name = row[1][0]  # Adjust this based on the actual structure
    #print(dataset_name)
    data = fetch_data(dataset_name)
    # Further processing if needed
    return data

# Fetch the filtered datasets in parallel
with ThreadPoolExecutor() as executor:
    datasets = list(executor.map(fetch_dataset, filtered_datasets.iterrows()))

# 'datasets' now contains the datasets that meet your criteria


In [24]:
type(datasets[20])

pandas.core.frame.DataFrame